In [1]:
import pandas as pd
import numpy as np
import subprocess
import random
from scipy import stats
import glob
import math
import csv
import sys
import os

import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
%matplotlib inline

In [ ]:
# begin by preprocessing the rda n R if not possible to load through python
#   from here need to get:
#      1. number of reads
#      2. TPM
#      3. transcript name
#      4. transcript type
#      5. gene type
# next need to get the TPM and 